# knapsack class implementation

In [14]:
import numpy as np

class Knapsack:
    
    def __init__(self, maxw, weights, values):
        if len(weights) != len(values):
            raise ValueError("Invalid lists lengths. the two lists should be the same size.")
        self.__maxw = maxw
        self.__weights = np.array(weights)
        self.__values = np.array(values)
        
    
    def setWeights(self, weights):
        """set weights list to the object"""
        self.__weights = np.array(weights)
    
    def setValues(self, values):
        """set values list to the object"""
        self.__values = np.array(values)
    
    def setMaxW(self, maxw):
        """set max withd of knapsack object"""
        self.__maxw = maxw
    
    def __createSortedTable(self, func, rev=False):
        """
            create sorted table with {func} condition and [rev] sort order
            return sorted table
        """
        table = [(self.__weights[i], self.__values[i]) for i in range(0, len(self.__weights))]
        table.sort(key=func, reverse=rev)
        return table
    
    
    def __getValues(self, table, zo = False):
        """
            return summation of first n item in {table} where n is 
            the number of lements that there summation <= max weight
            of the knapsack
            zo = True for 0-1 algorithm
        """
        mxval = 0
        curw = 0
        indx = 0
        while(indx < len(table) and curw + table[indx][0] <= self.__maxw):
            curw += table[indx][0]
            mxval += table[indx][1]
            indx +=1
        else:
            if indx < len(table) and not zo:
                remain = self.__maxw - curw
                mxval += table[indx][1] * (remain / table[indx][0])
        return mxval
    
    def getValuesWithMinimumWeights(self, zo=False):
        """
            return the maximum values starting from the minimum weights
            zo = True for 0-1 algorithm
        """
        return self.__getValues(self.__createSortedTable(lambda x: x[0]), zo)
    
    
    def getValuesWithMaximumValues(self, zo=False):
        """
            return the maximum values starting from the largest values
            zo = True for 0-1 algorithm
        """
        return self.__getValues(self.__createSortedTable(lambda x: x[1], rev=True), zo)
    
    
    def getValuesWithRatio(self, zo=False):
        """
            return the maximum values starting from the largest ratio
            between value / weights
            zo = True for 0-1 algorithm
        """
        return self.__getValues(self.__createSortedTable(lambda x: x[1]/x[0], rev=True), zo)
    
    def __str__(self):
        return "Weights List: " + str(self.__weights) + "\n" + "Values List: " + str(self.__values) +  "\n" + "Max Weight: " + str(self.__maxw)
    
    def __eq__(self, obj):
        if isinstance(obj, Knapsack):
            if self.__weights.tolist() == obj.__weights.tolist() and self.__values.tolist() == obj.__values.tolist() and self.__maxw == obj.__maxw:
                return True
            return False

# Main Application Class Controller:

In [15]:
class AppController:
    
    def __init__(self):
        self.knapsacks = []
        self.mainKnapsack = None
        self.zo = False
        self.currentMenu = self.printMainMenu
        
    def printKnapsacksList(self):
        if len(self.knapsacks) == 0:
            self.printError("There Is no Items To Print.")
            return
        
        for i in range(len(self.knapsacks)):
            self.printInfo(f"Knapsack Number {i+1}:")
            self.printInfo(self.knapsacks[i])
            self.printInfo("----------------------------")
    
    def printMainKnapsack(self):
        if self.mainKnapsack == None:
            self.printError("There Is no Main Knapsack Selected.")
        else:
            self.printInfo("The Main Knapsack:")
            self.printInfo(self.mainKnapsack)
            
            
    def printUsedAlgo(self):
        if self.zo == False:
            self.printInfo("Used Algorithm is: Fractional Algorithm")
        else:
            self.printInfo("Used Algorithm is: 1/0 Algorithm")
    
    def printError(self, message):
        print('\033[91m' + str(message) + '\033[0m')

    def printInfo(self, message):
        print('\033[92m' + str(message) + '\033[0m')
            
    
    def __readNkapsakData(self):
        weights = values = mx = None

        weightsGood = False
        while(not weightsGood):
            try:
                weights = list(map(float, input("Enter a Weights List: ").strip().split()))
                weightsGood = True
            except:
                self.printError("The list should be a list of numbers.")

        valuesGood = False
        while(not valuesGood):
            try:
                values = list(map(float, input("Enter a Values List: ").strip().split()))
                valuesGood = True
            except:
                self.printError("The list should be a list of numbers.")
        goodMax = False
        while(not goodMax):
            try:
                mx = float(input("Enter Max Weight for the knapsack: "))    
                goodMax = True
            except:
                self.printError("The Max should be a number.")        
        return (mx, weights, values)

    def createKnapsack(self, n=-1):
        mx, weights, values = self.__readNkapsakData()
        try:
            if n == -1:
                self.knapsacks.append(Knapsack(mx, weights, values))
                self.printInfo("Object Created.")
            else:
                self.knapsacks[n] = Knapsack(mx, weights, values)
                self.printInfo("Object Updated.")
        except:
            self.printError("Invalid lists lengths, the two lists should be the same size.")
            
    
    def readValidPosition(self):
        n = len(self.knapsacks)
        goodN = False
        while(not goodN):
            try:
                n = input("Enter The knapsack Number: ")
                n = int(n) - 1
                goodN = True
            except:
                self.printError("You should Enter a number.")
                
        if n >= len(self.knapsacks) or n < 0:
            self.printError(f"There is no object of Number ({n+1})")
            return -1
        return n
    
    def selectOperationType(self):
        n = 0
        algo = ["Fractional Algorithm", "0/1 Algorithm"]
        for i in range(len(algo)):
            print(str(i+1) + "- " + algo[i])
            
        goodN = False
        while(not goodN):
            try:
                n = input("Enter The Algorithm Number: ")
                n = int(n) - 1
                goodN = True
            except:
                self.printError("You should Enter a number.")
                
        if n >= len(algo) or n < 0:
            self.printError(f"Invalid Algorithm Number ({n+1})")
        else:
            self.zo = (n==1)
            self.printUsedAlgo()
    
    def updateKnapsack(self):
        n = self.readValidPosition()
        if n != -1:
            self.createKnapsack(n)
            
    def deleteKnapsack(self):
        n = self.readValidPosition()
        if n!= -1:
            if self.knapsacks[n] == self.mainKnapsack:
                self.mainKnapsack = None
            del self.knapsacks[n]
            self.printInfo("Object Deleted.")
    
    def selectMainKnapsack(self):
        n = self.readValidPosition()
        if n != -1:
            self.mainKnapsack = self.knapsacks[n]
            self.printInfo("The Main Object is Selected.")
    
    def profitFromMaxValue(self):
        if self.mainKnapsack == None:
            self.printError("There Is no Main Knapsack Selected Please Select One From the main Menu.")
        else:
            self.printInfo("The Maximum Profit from Maximizing Values: " + str(self.mainKnapsack.getValuesWithMaximumValues(self.zo)))
            
    def profitFromMinimumWeight(self):
        if self.mainKnapsack == None:
            self.printError("There Is no Main Knapsack Selected Please Select One From the main Menu.")
        else:
            self.printInfo("The Maximum Profit from Minimizing Weights: " + str(self.mainKnapsack.getValuesWithMinimumWeights(self.zo)))
            
    def profitFromMaxRatio(self):
        if self.mainKnapsack == None:
            self.printError("There Is no Main Knapsack Selected Please Select One From the main Menu.")
        else:
            self.printInfo("The Maximum Profit from Maximizing Ratio: " + str(self.mainKnapsack.getValuesWithRatio(self.zo)))
    
    def profitAll(self):
        if self.mainKnapsack == None:
            self.printError("There Is no Main Knapsack Selected Please Select One From the main Menu.")
        else:
            self.profitFromMaxValue()
            self.profitFromMinimumWeight()
            self.profitFromMaxRatio()

    def printMainMenu(self):
        self.currentMenu = self.printMainMenu
        optionsList = [self.createKnapsack, self.updateKnapsack, self.deleteKnapsack, self.selectMainKnapsack, 
                       self.printMainKnapsack, self.printKnapsacksList, self.printOperationsMenu, self.exit]
        menuList = ["Create Knapsack", "Update Knapsack", "Delete Knapsack", 
                    "Select Main Knapsack", "Print Main Knapsacks", "Print all Knapsacks",
                    "Knapsack Operations"]
        print(">>>>>>>>>>>Main Menu<<<<<<<<<<<")
        for i in range(len(menuList)):
            print(str(i+1)+ "- " + menuList[i])
        print("0- Exit")
        print(">>>>>>>>>>>>>>>|<<<<<<<<<<<<<<<")
        option = -1

        try:
            option = input("Choose Option:")
            option = int(option)
            if option > len(menuList) or option < 0:
                raise Exception("")
            optionsList[option-1]()
            if option == 0:
                return -1
            return 1
        except:
            app.printError(f"{option} is not valid Option.")

    def printOperationsMenu(self):
        self.currentMenu = self.printOperationsMenu
        optionsList = [self.selectOperationType, self.profitFromMaxValue, 
                       self.profitFromMinimumWeight, self.profitFromMaxRatio,
                       self.profitAll, self.printMainMenu]
        menuList = ["Select Algorithm Type", "Profit From Maximum value", "Profit From Minimum Weight", 
                    "Profit From Maximum Ratio", "Profit From All"] 
        print(">>>>>>>>>>>Operation Menu<<<<<<<<<<<")
        print("Choose Operation:")
        for i in range(len(menuList)):
            print(str(i+1)+ "- " + menuList[i])
        print("0- Back to Main Menu")
        print(">>>>>>>>>>>>>>>>>||<<<<<<<<<<<<<<<<<")
        option = 0

        try:
            option = input("Choose Option:")
            option = int(option)
            if option > len(menuList) or option < 0:
                raise Exception("")
            optionsList[option-1]()
            return 1
        except:
            app.printError(f"{option} is not valid Option.")

    def exit(self):
        print("App Closed.")
        
    def run(self):
        return self.currentMenu()
        
        

# Application Runner

In [17]:
running = True
app = AppController()
while(running):
    ret = app.run()
    if ret == -1:
        running = False

>>>>>>>>>>>Main Menu<<<<<<<<<<<
1- Create Knapsack
2- Update Knapsack
3- Delete Knapsack
4- Select Main Knapsack
5- Print Main Knapsacks
6- Print all Knapsacks
7- Knapsack Operations
0- Exit
>>>>>>>>>>>>>>>|<<<<<<<<<<<<<<<
Choose Option:0
App Closed.
